# Целевые переменные

- Целевые функции: realized volatility horizon, spike_flag, quantiles
- Анализ распределения target'ов, сохранение меток для обучения моделей

Критически важно для ML-задачи правильное создание и смещение целевых переменных!


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


## Функции создания целевых переменных


In [ ]:
def create_realized_vol_target(returns, horizon=5):
    """
    Целевая переменная: реализованная волатильность на горизонте
    Сдвиг на будущее для предсказания
    """
    # Рассчитываем стандартное отклонение на следующие 'horizon' дней
    target = returns.rolling(window=horizon).std().shift(-horizon) * np.sqrt(252)
    return target


def create_spike_flag(returns, threshold=2.0, window=20):
    """
    Бинарная целевая: будет ли всплеск волатильности
    1 = превышение порога, 0 = нормальное состояние
    """
    # Расчет z-score на будущий период
    future_returns = returns.shift(-1)
    ma = returns.rolling(window=window).mean()
    std = returns.rolling(window=window).std()
    z_score = np.abs((future_returns - ma) / std)
    
    spike = (z_score > threshold).astype(int)
    return spike


def create_quantile_targets(returns, horizon=5, quantiles=[0.16, 0.50, 0.84]):
    """
    Квантили распределения будущих доходностей
    Используется для quantile regression
    """
    targets = {}
    for q in quantiles:
        # Вычисляем квантиль на скользящем окне будущих данных
        target = returns.rolling(window=horizon).quantile(q).shift(-horizon)
        targets[f'quantile_{int(q*100)}'] = target
    
    return pd.DataFrame(targets)


def create_directional_target(returns, horizon=1):
    """
    Направление движения: 1 = вверх, 0 = вниз
    Для классификации
    """
    future_return = returns.shift(-horizon)
    direction = (future_return > 0).astype(int)
    return direction

print("✅ Функции целевых переменных загружены")


## Загрузка и создание целевых переменных


In [ ]:
# Загрузка данных
DATA_DIR = Path('data') / 'processed'
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")

# Создание целевых переменных
horizons = [1, 5, 10]

for h in horizons:
    df[f'target_vol_{h}d'] = create_realized_vol_target(df['log_return'], horizon=h)

# Спайк флаг
df['target_spike'] = create_spike_flag(df['log_return'], threshold=2.0, window=20)

# Квантили
quantile_df = create_quantile_targets(df['log_return'], horizon=5, quantiles=[0.16, 0.50, 0.84])
df = pd.concat([df, quantile_df], axis=1)

# Направление
df['target_direction'] = create_directional_target(df['log_return'], horizon=1)

print("✅ Целевые переменные созданы")
print(f"\nСтолбцы с целевыми:")
target_cols = [col for col in df.columns if col.startswith('target_') or col.startswith('quantile_')]
print(target_cols)


## Анализ распределения целевых переменных


In [ ]:
# Статистика целевых переменных
print("\n=== Статистика целевой волатильности (5 дней) ===")
print(df['target_vol_5d'].describe())

print("\n=== Распределение spike flag ===")
print(df['target_spike'].value_counts())
print(f"Доля всплесков: {df['target_spike'].mean():.2%}")

print("\n=== Распределение direction ===")
print(df['target_direction'].value_counts())
print(f"Доля роста: {df['target_direction'].mean():.2%}")

print("\n=== Квантили ===")
print(df[['quantile_16', 'quantile_50', 'quantile_84']].describe())


## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'features'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / f"{ticker}_with_targets.parquet"
df.to_parquet(output_path, index=False)
print(f"✅ Сохранено: {output_path}")
print(f"\n📊 Всего признаков: {len(df.columns)}")
